In [ ]:
from cnn_sliding_window_feats import extract_features
import networkx as nx
import nmslib
import numpy as np
import csv
from os import path

def prepare_image_paths_labels():
    labels_file = "household_images/labels.csv"
    data = genfromtxt(labels_file, delimiter=',')
    return data


def prepare_image_paths_labels():
    directory_path = "household_images/images"
    labels_file = "household_images/labels.csv"

    with open(labels_file, "rt", encoding="utf8") as f_input:
        csv_input = csv.reader(f_input)
        header = next(csv_input)
        data = [row for row in csv_input]
    
    image_paths = [path.join(directory_path, row[0]) for row in data]
    image_labels = [int(row[1]) for row in data]
    
    return image_paths, image_labels


image_paths, image_labels = prepare_image_paths_labels()
features = extract_features(image_paths[:1])


descriptors = []

for feature in features:
    descriptors.append(feature.cnn_features)
    
desc_index = nmslib.init(method='hnsw', space='cosinesimil')
desc_index.addDataPointBatch(np.array(descriptors), np.arange(len(descriptors)))
desc_index.createIndex({'post': 2}, print_progress=True)


nn_graph = nx.Graph()

for i in range(len(descriptors)):
    idxs, distances = desc_index.knnQuery(descriptors[i], k=17)
    for j in range(1,len(idxs)):
        nn_graph.add_edge(i, idxs[j], weight=distances[j])
        
for i in range(len(descriptors)):
    nn_graph.node[i]['image'] = features[i].image
    nn_graph.node[i]['coordinates'] = ' '.join(map(str, features[i].coordinates))
    nn_graph.node[i]['color'] = ' '.join(map(str, features[i].color)) # str(features[i].color[0], " ", features[i].color[1], " ", features[i].color[2])
    
nx.write_gml(nn_graph, "nn_graph_sample.gml", lambda x: str(x) )

In [ ]:
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt

block_grid=(21,28)

nn_graph = nx.read_gml("nn_graph.gml")

def visualize(segment_graph):
    group = 1
    thumbnail_img = np.zeros((block_grid[0], block_grid[1], 3))
    group_img = np.zeros(block_grid)

    for subgraph in nx.connected_component_subgraphs(segment_graph):
        for node in subgraph.nodes():
            if nn_graph.node[node]['image'] == nn_graph.node['0']['image']:
                coord = tuple(map(int, nn_graph.node[node]['coordinates'].split()))
                color = tuple(map(int, nn_graph.node[node]['color'].split()))
                group_img[coord] = group
                thumbnail_img[coord] = np.array(color)/255.0
        group += 1

    #print(thumbnail_img)
    plt.imshow(thumbnail_img[:,:,::-1])
    plt.matshow(group_img)
    plt.show()

    
def max_diff(nodes, segment_graph):
    # Degenerate case
    if len(nodes) == 1:
        return 0.0

    
    edge = max(segment_graph.edges(nodes,data=True), key=(lambda e: float(e[2]['weight'])))
    return float(edge[2]['weight'])


k = 0.285


def min_internal_diff(nodes1, nodes2, graph):
    global k
    tauC1 = k/len(nodes1)
    tauC2 = k/len(nodes2)

    return min(max_diff(nodes1, graph) + tauC1, max_diff(nodes2, graph) + tauC2)


def segment(segment_graph, sorted_edges):
    for u,v,d in sorted_edges:
        connected1 = nx.node_connected_component(segment_graph, u)
        connected2 = nx.node_connected_component(segment_graph, v)
        w = min_internal_diff(connected1, connected2, segment_graph)
        if float(d['weight']) <= w:
            segment_graph.add_edge(u,v,weight=float(d['weight']))
    return segment_graph


segment_graph = nx.Graph()
segment_graph.add_nodes_from(nn_graph.nodes())
sorted_edges = sorted(nn_graph.edges(data=True), key=(lambda e: float(e[2]['weight'])))

segment_graph = segment(segment_graph, sorted_edges)
visualize(segment_graph)

In [ ]:
visualize(segment_graph)

In [ ]:
from cnn_sliding_window_feats import open_and_prepare_image
from matplotlib import pyplot as plt

window_size_blocks = 7 
block_size=32
block_grid=(21,28)

window_size = window_size_blocks * block_size
steps = (block_grid[0]-window_size_blocks+1, block_grid[1]-window_size_blocks+1)
stride = block_size
    
img = open_and_prepare_image(image_path, steps=steps)
img2 = img[:,:,::-1]
print(img.shape)
print(img2.shape)
plt.imshow(img2)

# (height, width)

In [ ]:
from cnn_sliding_window_feats import extract_windows
from matplotlib import pyplot as plt

image_path = "household_images/images/IMG_0412.jpg"

window_size_blocks = 7 
block_size=32
block_grid=(21,28)

window_size = window_size_blocks * block_size
steps = (block_grid[0]-window_size_blocks+1, block_grid[1]-window_size_blocks+1)
stride = block_size

windows, coords, thumbnail = extract_windows(image_path, steps=steps)
img2 = thumbnail[:,:,::-1]/255
plt.imshow(img2)

In [ ]:
np.array((1,2,3))

In [ ]:

tuple(map(int, np.array((1.0,2.0,3.0))))

In [ ]:
import numpy as np
edges = np.zeros(shape=(3, 3), dtype=object)

In [ ]:
edges[0,:] = [1,2,3]

In [ ]:
edges